In [ ]:
from collections import defaultdict
import copy
import random
import os
import shutil
from urllib.request import urlretrieve

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models

cudnn.benchmark = True

In [ ]:
class TqdmUpTo(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)


def download_url(url, filepath):
    directory = os.path.dirname(os.path.abspath(filepath))
    os.makedirs(directory, exist_ok=True)
    if os.path.exists(filepath):
        print("Filepath already exists. Skipping download.")
        return

    with TqdmUpTo(unit="B", unit_scale=True, unit_divisor=1024, miniters=1, desc=os.path.basename(filepath)) as t:
        urlretrieve(url, filename=filepath, reporthook=t.update_to, data=None)
        t.total = t.n


def extract_archive(filepath):
    extract_dir = os.path.dirname(os.path.abspath(filepath))
    shutil.unpack_archive(filepath, extract_dir)

In [ ]:
dataset_directory = "../datasets/cats-vs-dogs"

In [ ]:
def plot(image):
    plt.imshow(image)
    plt.show()

In [ ]:
# filepath = os.path.join(dataset_directory, "kagglecatsanddogs_3367a.zip")
# download_url(
#     url="https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip",
#     filepath=filepath,
# )
# extract_archive(filepath)

In [ ]:
root_directory = os.path.join(dataset_directory, "PetImages")

cat_directory = os.path.join(root_directory, "Cat")
dog_directory = os.path.join(root_directory, "Dog")

cat_images_filepaths = sorted([os.path.join(cat_directory, f) for f in os.listdir(cat_directory)])
dog_images_filepaths = sorted([os.path.join(dog_directory, f) for f in os.listdir(dog_directory)])
images_filepaths = [*cat_images_filepaths, *dog_images_filepaths]
correct_images_filepaths = [i for i in images_filepaths if cv2.imread(i) is not None]

random.seed(42)
random.shuffle(correct_images_filepaths)
train_images_filepaths = correct_images_filepaths[:20000]
val_images_filepaths = correct_images_filepaths[20000:-10]
test_images_filepaths = correct_images_filepaths[-10:]
print(len(train_images_filepaths), len(val_images_filepaths), len(test_images_filepaths))

In [ ]:
print("len dog_images_filepaths:", len(dog_images_filepaths))
print("len cat_images_filepaths:", len(cat_images_filepaths))
print("len images_filepaths:", len(images_filepaths))

In [ ]:
print(dog_images_filepaths[1:4])

In [ ]:
def display_image_grid(images_filepaths, predicted_labels=(), cols=5):
    rows = len(images_filepaths) // cols
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 6))
    for i, image_filepath in enumerate(images_filepaths):
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        true_label = os.path.normpath(image_filepath).split(os.sep)[-2]
        predicted_label = predicted_labels[i] if predicted_labels else true_label
        color = "green" if true_label == predicted_label else "red"
        ax.ravel()[i].imshow(image)
        ax.ravel()[i].set_title(predicted_label, color=color)
        ax.ravel()[i].set_axis_off()
    plt.tight_layout()
    plt.show()

In [ ]:
display_image_grid(test_images_filepaths)

In [ ]:
class CatsVsDogsDataset(Dataset):
    def __init__(self, images_filepaths, transform=None):
        self.images_filepaths = images_filepaths
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
        # plot(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if os.path.normpath(image_filepath).split(os.sep)[-2] == "Cat":
            label = 1.0
        else:
            label = 0.0
        if self.transform is not None:
            image = self.transform(image=image)["image"]

        plot(image.permute(1, 2, 0))
        return image, label

In [ ]:
params = {
    "model": "resnet50",
    "device": "cuda",
    "lr": 0.001,
    "batch_size": 64,
    "num_workers": 4,
    "epochs": 10,
}

In [ ]:
train_transform = A.Compose(
    [
        # A.SmallestMaxSize(max_size=160),
        A.ShiftScaleRotate(shift_limit=0.01, scale_limit=0.01, rotate_limit=90),
        # A.RandomCrop(height=, width=128),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        # A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.RandomRotate90(),
        ToTensorV2(),
    ]
)
train_dataset = CatsVsDogsDataset(images_filepaths=train_images_filepaths[:10], transform=train_transform)
train_dataset[0]
train_loader = DataLoader(
    train_dataset, batch_size=params["batch_size"], shuffle=True, num_workers=params["num_workers"], pin_memory=True,
)

In [ ]:
image_filepath = "../DATN_data/test/2_acefalgan_(00)__(1)_crop.jpg"
import albumentations as A
from albumentations.pytorch import ToTensorV2
image = cv2.imread(image_filepath)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plot(image)
image_shape = image.shape[:2]

In [ ]:
xa_transform = A.Compose(
    [
        A.CropAndPad(px = 30, keep_size = False, pad_mode= 4, p = 0.5),
        # A.SmallestMaxSize(max_size=160),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=45, p = 0.5),
        # A.RandomCrop(height= round(image_shape[0]*0.9), width= round(image_shape[1]*0.9)),
        A.RGBShift(r_shift_limit=5, g_shift_limit=5, b_shift_limit=5, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        # A.MultiplicativeNoise(multiplier=0.5, p=0.5),
        # A.MultiplicativeNoise(multiplier=1.5, p=0.5),
        A.Blur(blur_limit=(5,5), p=0.5),
        # A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        # A.Resize(height=image_shape[0], width=image_shape[1]),
        ToTensorV2(),
    ]
)





# for i in range(20):
#     plot(xa_transform(image=image)["image"])



In [ ]:
xa_transform = A.Compose(
    [
        A.CropAndPad(px = 20, keep_size = False, pad_mode= 2, p = 1),

        A.CropAndPad(px = 20, keep_size = False, pad_mode= 1, p = 1),

        # # A.SmallestMaxSize(max_size=160),
        # A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=45, p = 0.5),
        # # A.RandomCrop(height= round(image_shape[0]*0.9), width= round(image_shape[1]*0.9)),
        # A.RGBShift(r_shift_limit=5, g_shift_limit=5, b_shift_limit=5, p=0.5),
        # A.RandomBrightnessContrast(p=0.5),
        # # A.MultiplicativeNoise(multiplier=0.5, p=0.5),
        # # A.MultiplicativeNoise(multiplier=1.5, p=0.5),
        # A.Blur(blur_limit=(5,5), p=0.5),
        # # A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        # # A.Resize(height=image_shape[0], width=image_shape[1]),
        ToTensorV2(),
    ]
)

print("image type:", type(image))
print("image shape:", image.shape)
img_ex = xa_transform(image=image)["image"]
type(img_ex)


# for i in range(10):
#     plot(xa_transform(image=image)["image"])

In [ ]:
type(xa_transform)